# SharedTask Touche23 Human Value Detection

## Written by Madeleine Wallace and John Ortiz

### Bert Multi-Classification Problem :
-  base code is representative from https://colab.research.google.com/github/abhimishra91/transformers-tutorials/blob/master/transformers_multi_label_classification.ipynb

In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

Re-Importing

In [2]:
file = open("data/arguments-training.tsv", 'r', encoding='utf8')
x_train = [line.strip().split('\t') for line in file.readlines()[1:]]
file.close()
print(x_train[0])

file = open("data/labels-training.tsv", 'r', encoding='utf8')
y_train = [line.strip().split('\t') for line in file.readlines()[1:]]
file.close()

file = open("data/arguments-validation.tsv", 'r', encoding='utf8')
x_valid = [line.strip().split('\t') for line in file.readlines()[1:]]
file.close()
print(x_valid[0])
file = open("data/labels-validation.tsv", 'r', encoding='utf8')
y_valid = [line.strip().split('\t') for line in file.readlines()[1:]]
file.close()
print(y_valid[0])
file = open("data/arguments-test.tsv", 'r', encoding='utf8')
x_test = [line.strip().split('\t') for line in file.readlines()[1:]]
file.close()
print(x_test[0])

['A01002', 'We should ban human cloning', 'in favor of', 'we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same.']
['A01001', 'Entrapment should be legalized', 'in favor of', "if entrapment can serve to more easily capture wanted criminals, then why shouldn't it be legal?"]
['A01001', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
['A26004', 'We should end affirmative action', 'against', 'affirmative action helps with employment equity.']


Retokenize for BERT optimization

In [3]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.args = dataframe.args
        self.targets = self.data.labs

        special_tokens = ["<PRO>", "<CON>"] #-
        self.tokenizer.add_tokens(special_tokens, special_tokens = True) #-


    def __len__(self):
        return len(self.args)

    def __getitem__(self, index):
        args = str(self.args[index])
        args = " ".join(args.split())

        inputs = self.tokenizer.encode_plus(
            args,
            None,
            add_special_tokens=True,
            padding='max_length',
            max_length = 200 ,#len(max(self.args)) #--------------------------------------------------------changed here
            pad_to_max_length = True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        self.targets[index] = [int(l) for l in self.targets[index]]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.long)
        }

In [4]:
from typing import Dict, List
def combine(text, labels=None):
    
    args = []
    labs = []
    if(labels != None):
        for arg, lab in zip(text, labels):
            if arg[3] == 'in favor of':
                sep = '<PRO>'
            else:
                sep = '<CON>'
            item = '<SOS>' + arg[1] + sep + arg[3] + '<EOS>'
            args.append(item)
            labs.append(lab[1:20])
    else:
        print("ValueError: labels can not be noneType Object")
    print(len(args),len(labs))
    combined = pd.DataFrame(data = {'args': args, 'labs': labs})
    return combined




In [5]:
#y_train[0]
len(x_train[:1000])

1000

In [18]:

train_comb = combine(x_train[:1000], y_train[:1000])
valid_comb = combine(x_valid[:1000], y_valid[:1000])


for lab in train_comb["labs"]:
    lab = [int(l) for l in lab]

for lab in valid_comb["labs"]:
    lab = [int(l) for l in lab]


1000 1000
1000 1000


In [19]:
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

training_set = CustomDataset(train_comb, tokenizer)
testing_set = CustomDataset(valid_comb, tokenizer)

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)


In [27]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        #config = BertConfig(hidden_size = 256)
        #self.l1 = self.l1(config)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 19)   # 768
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids,return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [9]:
import tqdm

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

def train(epoch,model):
    model.train()
    for _,data in tqdm.tqdm(enumerate(training_loader, 0)): #------------------------------------TRAIN X CALLED HERE
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Progress {_} || Epoch: {epoch}, Loss:  {loss.item()}')
            
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return model

In [29]:
for epoch in range(EPOCHS):
    model = train(epoch,model)

0it [00:10, ?it/s]


KeyboardInterrupt: 

In [10]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [19]:
outputs, targets = validation(epoch)
outputs = np.array(outputs) >= 0.5
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.003
F1 Score (Micro) = 0.0
F1 Score (Macro) = 0.0


In [21]:
train_comb = combine(x_train, y_train)
valid_comb = combine(x_valid, y_valid)


for lab in train_comb["labs"]:
    lab = [int(l) for l in lab]

for lab in valid_comb["labs"]:
    lab = [int(l) for l in lab]

TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 8
EPOCHS = 3
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

training_set = CustomDataset(train_comb, tokenizer)
testing_set = CustomDataset(valid_comb, tokenizer)

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)


#for epoch in range(EPOCHS):
#    model = train(epoch,model)

5393 5393
1896 1896


In [24]:
for epoch in range(EPOCHS):
    model = train(epoch,model)

0it [00:25, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x768 and 288x19)

In [ ]:
torch.cuda.empty_cache()

In [13]:
class BERTClass2(torch.nn.Module):
    def __init__(self):
        super(BERTClass2, self).__init__()
        config = BertConfig(hidden_size = 288)
        self.l1 = transformers.BertModel(config).from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(288, 19)   # 768
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids,return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

model2 = BERTClass2()
model2.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass2(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
   

In [14]:
for epoch in range(EPOCHS): 
    model2 = train(epoch,model2) 

0it [00:22, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x768 and 288x19)